### StructuredOutputParser
* LLM 나오는 답변을 dict 혀식으로 구성을 해준다.
* key/value 쌍으로 필드를 반환하고 싶을때 사용할 수 있다.
* Gpt-4 또는 Calude 모델보다 파라미터수가 적은 로컬도멜들 같은 경우에 효과적이다.

In [2]:
from dotenv import load_dotenv

load_dotenv()

True

In [3]:
from langchain.output_parsers import ResponseSchema, StructuredOutputParser

response_schema = [
    ResponseSchema(name="answer", description="사용자 질문에 대한 답변"),
    ResponseSchema(
        name="source",
        description="사용자의 질문에 답하기 위해 사용된 '출처'. '웹사이트주소' 이어야 한다."
    )
]

# 응답 스키마를 기반으로 .StructuredOutputParser 생성
output_parser = StructuredOutputParser.from_response_schemas(response_schema)

In [5]:
from  langchain_core.prompts import PromptTemplate

format_instructions = output_parser.get_format_instructions()

print(format_instructions)

prompt = PromptTemplate(
    template = "answer the question a s posible \n Format\n{format_instructions} \nQuestion\n{question}",
    input_variables=["question"],
    partial_variables={"format_instructions" : format_instructions}
)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"answer": string  // 사용자 질문에 대한 답변
	"source": string  // 사용자의 질문에 답하기 위해 사용된 '출처'. '웹사이트주소' 이어야 한다.
}
```


In [8]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(temperature=0, model="gpt-4o")
chain = prompt | model | output_parser

In [12]:
chain.invoke({"question" : "미국 명소 알려줘"})

{'answer': '미국에는 다양한 명소가 있습니다. 그 중 몇 가지를 소개하자면, 뉴욕의 자유의 여신상, 그랜드 캐니언 국립공원, 샌프란시스코의 금문교, 라스베이거스의 스트립, 그리고 워싱턴 D.C.의 국회의사당 등이 있습니다.',
 'source': 'https://www.travelandleisure.com/worlds-best/landmarks-in-the-us'}